### Выбранные стили: y2k vs Classic

1. По 20 изображений каждого стиля сохранениы в папке `data`:
    - `/style_1` - y2k
    - `/style_2` - classic

2. **y2k**: яркий, броский, насыщенный стиль, характерный для моды конца 90-х – начала 2000-х. Он отличается обилием блестящих материалов, металлизированных поверхностей, трикотажа с эластичностью, количеством открытых элементов, низкой посадкой брюк, обилием аксессуаров и насыщенной цветовой палитрой. Силуэты часто акцентирующие фигуру как засчет только приталенности, так и засчет сочетания ее с объемными вещам. Текстуры включают глянец, деним, кожу, меховые детали и принты. Обувь часто на высоком тонком каблуке или же акцентная спортивная. Характерен для тинейджеров.
   
   **classic**: характерны минимализм и структурированность, актуальность сквозь года. Этот стиль использует качественные натуральные материалы: шерсть, хлопок, кашемир, кожу. А также нейтральные и приглушённые цвета: чёрный, белый, бежевый, тёмно-синий. Силуэты чаще прямые, строгие и симметричные: пиджаки, брюки со стрелками, гладкие платья, пальто. Текстуры матовые, без лишнего декора, с чистыми линиями и чёткой посадкой. Количество аксессуаров минимально. Обувь зачастую из кожи на сдержанном каблуке. Может быть присущ людям всех возрастов.